In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import ot
import pandas as pd
from domain_model import StateVariable
from databaseemulator import DataBaseEmulator
from stats.fastkde import KDE

In [ ]:
# Load the database with the cut-in scenarios.
filename = os.path.join("data", "5_scenarios", "cut_in_scenarios.json")
cutins = DataBaseEmulator(filename)
print("Number of scenarios: {:d}".format(len(cutins.collections["scenario"])))

In [ ]:
def cutin_parameters(scenario):
    # Obtain the duration of the lane change.
    activity = next(activity for activity in scenario.activities 
                    if activity.activity_category.name == 'lane change target')
    duration = activity.tduration
    ystart = activity.get_state(time=scenario.time["start"])[0]
    yend = activity.get_state(time=scenario.time["end"])[0]
    tstart = activity.tstart
    from_direction = 'r' if activity.name == "left lane change" else 'r'
    
    # Obtain longitudinal speed/position at time of start cut in.
    vstart, vend = None, None
    xstart = None
    init_activity_target = ''
    for activity in scenario.activities:
        if activity.activity_category.state == StateVariable.LON_TARGET and \
                activity.tstart <= scenario.time["start"] <= activity.tend:
            vstart = activity.get_state(time=scenario.time["start"])[0][0]
            vend = activity.get_state(time=activity.tend)[0][0]
            xstart = activity.get_state(time=scenario.time["start"])[1][0]
            if activity.activity_category.name == "deceleration target":
                init_activity_target = 'd'
            elif activity.activity_category.name == "acceleration target":
                init_activity_target = 'a'
            else:
                init_activity_target = 'c'
            tstartlon = activity.tstart - scenario.time["start"]
            tendlin = activity.tend - scenario.time["start"]
            break
            
    # Obtain ego vehicle speed at time of start cut in.
    vego = None
    for actor, activity, _ in scenario.acts:
        if actor.name == "ego vehicle" and \
            activity.activity_category.state == StateVariable.SPEED and \
                activity.tstart <= scenario.time["start"] <= activity.tend:
            vego = activity.get_state(time=scenario.time["start"])[0]
            
    return [duration, ystart, yend, xstart, vstart, vend, vego,
            from_direction, init_activity_target, tstartlon, tendlin]

In [ ]:
class CutInGenerator:
    def __init__(self, database):
        self.nscenarios = len(database.collections["scenario"])
        
        # Obtain various parameters for later use.
        df = pd.DataFrame([cutin_parameters(database.get_item("scenario", i)) 
                           for i in range(self.nscenarios)], 
                          columns=['duration', 'ystart', 'yend', 'xstart', 'vstart', 'vend', 'vego',
                                   'from_direction', 'init_activity_target', 'tstartlon', 'tendlon'])
        df["vdiff"] = df["vstart"] - df["vego"]
        df["ystartabs"] = np.abs(df["ystart"])
        df["yspeed"] = (df["yend"] - df["ystart"]) / df["duration"]
        df["yspeedabs"] = np.abs(df["yspeed"])
        df["tstartlonmap"] = np.log(-df["tstartlon"]+1)
        self.df = df
        
        # Determine the probabilities of the initial activities.
        # [prob of accelerating, prob of decelerating, prob of cruising]
        self.prob_init_activity = np.array([np.sum(df["init_activity_target"] == "a"),
                                            np.sum(df["init_activity_target"] == "d"),
                                            np.sum(df["init_activity_target"] == "c")])
        self.prob_init_activity = self.prob_init_activity / self.nscenarios
        
        # Construct the KDEs for the initial parameters.
        self.kdes = dict()
        self.kdes["ystart"] = KDE(data=df["ystart"].values, scaling=True)
        for act in ['a', 'd', 'c']:
            self.kdes["{:s}_xstart".format(act)] = KDE(data=df.loc[df["init_activity_target"] == act, 
                                                                   "xstart"].values,
                                                       scaling=True)
            self.kdes["{:s}_tstartlon".format(act)] = KDE(data=df.loc[df["init_activity_target"] == act, 
                                                                      "tstartlon"].values,
                                                          scaling=True)
            self.kdes["{:s}_speeds".format(act)] = KDE(data=df.loc[df["init_activity_target"] == act,
                                                                      ["vstart", "vego"]].values,
                                                       scaling=True)
        
        # Construct the KDEs for the parameters of the initial activities.
        self.kdes["duration"] = KDE(data=df["duration"])
        self.kdes["yend"] = KDE(data=df["yend"])
        for act in ['a', 'd', 'c']:
            self.kdes["{:s}_tendlon".format(act)] = \
                KDE(data=df.loc[df["init_activity_target"] == act, "tendlon"].values,
                    scaling=True)
            self.kdes["{:s}_vend".format(act)] = \
                KDE(data=df.loc[df["init_activity_target"] == act, ["vstart", "vend"]].values,
                    scaling=True)
        
        # Compute the bandwidth for all KDEs.
        for _, kde in self.kdes.items():
            kde.compute_bandwidth()
            
        # Compute 'transition' matrix
        A = np.zeros((3, 3))  # 'a', 'd', 'c'
        for i in range(self.nscenarios):
            s = database.get_item("scenario", i)
            t_acts = []
            lon_acts = []
            for actor, activity, tstart in s.acts:
                if actor.actor_category.name == "cut-in vehicle":
                    if activity.activity_category.state == StateVariable.LON_TARGET:
                        t_acts.append(tstart)
                        lon_acts.append(activity)
            if len(t_acts) <= 1:
                continue
            i_sorted = np.argsort(t_acts)
            for i, j in zip(i_sorted[:-1], i_sorted[1:]):
                ia = (0 if lon_acts[i].activity_category.name == "acceleration target" else
                      1 if lon_acts[i].activity_category.name == "deceleration target" else 2)
                ja = (0 if lon_acts[j].activity_category.name == "acceleration target" else
                      1 if lon_acts[j].activity_category.name == "deceleration target" else 2)
                A[ia, ja] += 1
        
    def generate(self):
        while True:
            # Step 1: Determine initial longitudinal activity.
            first_lon = ['a', 'd', 'c'][np.argmax(np.random.rand() < np.cumsum(self.prob_init_activity))]

            # Step 2: Initial parameters.
            ystart = self.kdes["ystart"].sample()[0][0]
            xstart = self.kdes["{:s}_xstart".format(first_lon)].sample()[0][0]
            tstartlon = self.kdes["{:s}_tstartlon".format(first_lon)].sample()[0][0]
            vstart, vego = self.kdes["{:s}_speeds".format(first_lon)].sample()[0]
            if tstartlon >= 0 or vstart <= 0 or vego <= 0 or np.abs(ystart) < 1:
                continue

            # Step 3: Parameters of initial activities.
            duration = self.kdes["duration"].sample()[0][0]
            yend = self.kdes["yend"].sample()[0][0]
            tendlon = self.kdes["{:s}_tendlon".format(first_lon)].sample()[0][0]
            kde = self.kdes["{:s}_vend".format(first_lon)]
            cdf_vend = (kde.cdf(np.vstack((np.ones(100)*(vstart+1e-4), np.linspace(0, 40, 100))).T) -
                        kde.cdf(np.vstack((np.ones(100)*vstart, np.linspace(0, 40, 100))).T)) / 1e-4
            vend = np.interp(np.random.rand()*cdf_vend[-1], cdf_vend, np.linspace(0, 40, 100))
            
            if tendlon <= 0 or (vstart >= vend and first_lon == 'a') or \
                    (vstart <= vend and first_lon == 'd') or vend <= 0 or duration < 1:
                continue
            
            # We have succesfully generated a test case!
            break
        
        return first_lon, duration, tendlon, vstart, vend
        
CIG = CutInGenerator(cutins)
CIG.prob_init_activity
CIG.generate()

In [ ]:
CIG.generate()

In [ ]:
v0, v1 = CIG.kdes["a_speeds"].sample()[0]

In [ ]:
v0

In [ ]:
x = np.array([[1, 2], [0, 0], [2, 0]])
y = np.array([[0, 2], [1, 1], [2, 2]])
M = ot.dist(x,y, metric='euclidean')

In [ ]:
n = 200
n2 = 400
ot.emd2(np.ones(n)/n, np.ones(n2)/n2, np.random.rand(n, n2))

In [ ]:
1 + np.sqrt(2)/3